# Semantic Search Word2Vec Model First Draft

This Jupyter notebook is meant to serve as an introduction to reading Github `.md` documentation and analyzing it...

## Phase 1: Documentation Data Reading and Pre-Processing

### Step 1: Reading and Storing the Documentation Data

In this section, we'll read the markdown `.md` file data, collect it, and store it for processing. We can do this by reading through all of the `.md` files in a directory and reading them into plain text format, then storing it.

In [13]:
import doc_reader as reader

doc_data = reader.collect_doc_data("docs/docs")

### Step 2: Cleaning the Documentation Data

In this section, we'll take our collected and stored documentation data from Step 1 and clean it up so we can use it. This could include removing HTML tags, removing punctuation and special characters, removing extra whitespaces from the text, making all of our text lowercase for semantic searching, and catching any mispellings in the documentation.

In [14]:
import md_cleaner as cleaner

cleaned_doc_data = cleaner.clean_doc_data(doc_data)

### Step 3: Pre-processing the Documentation Data

In this section, we'll take our cleaned documentation data from Step 2 and pre-process it by tokenization, stemming lemmatization, and stop-word removal.

In [15]:
import md_preprocessor as preprocessor

preproc_docs = preprocessor.preprocess_doc_data(cleaned_doc_data)

In [16]:
sum = 0
for doc, content in preproc_docs.items():
    sum += len(content)

print(sum)

11855


## Phase 2: Implementing Semantic Search with Word2Vec

Now, we can use `Gensim` to implement the semantic searching of the cleaned and pre-processed documentation data with its Word2Vec algorithm. This basically maps words and phrases to dense vector representations in a high-dimensional space.

In [17]:
import gensim.downloader

pretrained_model = gensim.downloader.load('word2vec-google-news-300')

In [18]:
import gensim
from gensim.models import Word2Vec
import numpy as np

corpus = list(preproc_docs.values())
model = Word2Vec(corpus, vector_size=500, window=5, min_count=5, workers=4)
model.build_vocab_from_freq(pretrained_model.key_to_index, corpus_count=len(corpus), update=True)

In [19]:
document_embeddings = {}

for filename, tokens in preproc_docs.items():
    embeddings = [model.wv[word] for word in tokens if word in model.wv]
    if embeddings:
        document_embeddings[filename] = gensim.matutils.unitvec(np.mean(embeddings, axis=0))

In [20]:
def run_query(query_str: str):
    """
    Performs a similarity search on the inputted query against the given Word2Vec model.

    Parameters:
        query_str (str) : user query
    
    Returns:
        similar_docs (list[str]) : most similar docs in order
    """
    corrected_query = cleaner.correct_spelling(query_str)
    query_tokens = preprocessor.preprocess_str(cleaner.clean_str(corrected_query))
    average_vec_rep = [model.wv[token] for token in query_tokens if token in model.wv]
    query_embedding = gensim.matutils.unitvec(np.mean(average_vec_rep, axis=0))

    similar_docs = []
    for filename, doc_embedding in document_embeddings.items():
        similarity_score = np.dot(doc_embedding, query_embedding)
        similar_docs.append((filename, similarity_score))
    
    similar_docs = sorted(similar_docs, key=lambda x: x[1], reverse=True)

    return similar_docs

In [21]:
def get_relevant_files(query, top_k=5, include_score=False, verbose=False):
    """
    Gets the top 'k' relevant files from an inputted query. Defaults to top
    5 most relevant files.

    Parameters:
        query (str) : question to search PW documentation for
        top_k (int) : top 'k' most relevant files to return (default: 5)
        include_score (bool) : if True, includes similarity score of file
        verbose (bool) : if True, prints files in addition to returning
    
    Returns:
        rel_files (list) : top 'k' most relevant files
    """
    try:
        similar_docs = run_query(query)
    except TypeError:
        print("Your query does not match anything in our system.")
        return []

    if include_score:
        rel_files = similar_docs[:top_k]
        if verbose:
            print(f"Top {top_k} most relevant files to your query with similarity scores included:\n")
            for i, (file, sim_score) in enumerate(rel_files):
                print(f"{i + 1}. {file}: {sim_score}")
        return rel_files
    else:
        rel_files = [filename for filename, _ in similar_docs[:top_k]]
        if verbose:
            print(f"Top {top_k} most relevant files to your query:\n")
            for i, file in enumerate(rel_files):
                print(f"{i + 1}. {file}")
    return rel_files

In [22]:
def get_file_content_from_filenames(filenames: list, docs: dict) -> list:
    """
    Helper function that takes a list of filenames and returns a list of
    the cleaned and preprocessed contents of those files.

    Parameters:
        filenames (list) : filenames to get content of
        docs (dict) : documents keyed by filename and valued with content

    Returns:
        file_content (list) : content of each of the inputted filenames 
    """
    return [docs[file] for file in filenames]

In [23]:
# for i, val in enumerate(preproc_docs.values()):
#     print(f"{i}. {len(val)}")

In [29]:
import openai
import helper_funcs as helper

def run_gpt(query, *args, api_key=helper.get_api_key()):
    """
    Function that runs the gpt-3.5-turbo AI API on a query and set of arguments
    Arguments should consist of a variable length list, where each
    element contains a list of tokens from the most relevant files related to
    the inputted query.

    Paramaters:
        query (str) : inputted query from user
        *args (list[list[str]]) : array containing document information tokens
        api_key (str) : user API key to run
    
    Returns:
        reply (str) : GPT AI response to query with supporting relevant documents
    """
    openai.api_key = api_key

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": query}
    ]

    for tokens in args:
        messages.append({"role": "user", "content": tokens})

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    reply = response.choices[0].message.content
    return reply

In [30]:
query = "Where can I find AWS key buckets?"

rel_files = get_relevant_files(query)

rel_content = get_file_content_from_filenames(rel_files, preproc_docs)

In [32]:
response = run_gpt(query, rel_content)
print(f"ChatGPT: {response}")

InvalidRequestError: [['transferring', 'data', 'tofrom', 'aws', 's3', 'storage', 'working', 'aws', 'cli', 'transfer', 'data', 'tofrom', 's3', 'bucket', 'storage', 'youll', 'use', 'aws', 'command', 'line', 'interface', 'cli', 'aws', 'cli', 'preinstalled', 'aws', 'cluster', 'pw', 'platform', 'enter', 'command', 'directly', 'ide', 'logging', 'controller', 'active', 'aws', 'cluster', 'youre', 'transferring', 'data', 's3', 'bucket', 'folder', 'onpremise', 'cluster', 'remote', 'device', 'youll', 'likely', 'need', 'install', 'aws', 'cli', 'first', 'check', 'aws', 'cli', 'open', 'terminal', 'command', 'line', 'navigate', 'data', 'destination', 'enter', 'aws', 'aws', 'cli', 'installed', 'youll', 'see', 'message', 'show', 'location', 'usrlocalbinaws', 'aws', 'cli', 'installed', 'youll', 'see', 'message', 'usrbinwhich', 'aws', 'aws', 'found', 'install', 'aws', 'cli', 'need', 'install', 'aws', 'cli', 'recommend', 'following', 'aws', 'installation', 'guide', 'includes', 'osspecific', 'instruction',